In [2]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
import h5py
import sys
import seqload

from tqdm import tqdm
from numpy import random
from scipy.stats import pearsonr, spearmanr

hist = lambda a, bins: np.histogram(a, bins=bins, density=True)[0].tolist()

# npos = int(
#     sys.argv[1]
# )  # this is the singular marginal order this file runs, from 2 to 14...
# reps = int(sys.argv[2])  # usually 500

# modelseqs = seqload.loadSeqs(sys.argv[4], "ABCD")[0]
# indepseqs = seqload.loadSeqs(sys.argv[5], "ABCD")[0]
# targetseqs = seqload.loadSeqs(sys.argv[3], "ABCD")[0]

repstack, su, fd2, fp2, fi2 = [], [], [], [], []

npos, reps = 4, 500
modelseqs = seqload.loadSeqs("../seq/gen_sVAE.seqs", "ABCD")[0]
indepseqs = seqload.loadSeqs("../seq/gen2.indep.seqs", "ABCD")[0]
targetseqs = seqload.loadSeqs("../seq/6M_test.seqs", "ABCD")[0]

targetseqs = (targetseqs + ord("A")).view("S1")
modelseqs = (modelseqs + ord("A")).view("S1")
indepseqs = (indepseqs + ord("A")).view("S1")

L = targetseqs.shape[1]
Nd = targetseqs.shape[0]
Np = modelseqs.shape[0]
Ni = indepseqs.shape[0]

allseq = np.concatenate([targetseqs, modelseqs, indepseqs])
choose_from = np.arange(0, L, 1)
choose_from = choose_from.astype("int")


repstack, su, fd2, fp2, fi2 = [], [], [], [], []


for i in tqdm(range(reps)):
    chosenpos = random.choice(choose_from, npos, replace=False)
    pos_string = int("".join([str(i + 1) for i in chosenpos]))

    # compute marginals (both datasets together, to we get counts for
    # subsequences that only appear in one datasets)
    # Numpy has no clear way to compute subsequence frequencies, so we
    # use a trick involving viewing the memory with a V(oid) datatype.
    v = np.ascontiguousarray(allseq[:, chosenpos]).view("S{}".format(npos))
    u, c = np.unique(v, return_inverse=True)
    bins = np.arange(len(u) + 1) - 0.5
    fd = hist(c[:Nd], bins)
    fp = hist(c[Nd : Nd + Np], bins)
    fi = hist(c[Nd + Np : Nd + Np + Ni], bins)

    fd2 += fd
    fp2 += fp
    fi2 += fi

    repstack += [i + 1 for n in range(0, len(u))]
    su += [
        int("".join([str("ABCD".index(ss)) for ss in (str(s).split("'")[1])]))
        for s in u
    ]
    top20 = np.argsort(fd)[-20:]
    fd20 = [fd[m] for m in top20]
    fp20 = [fp[m] for m in top20]
    fi20 = [fi[m] for m in top20]


result = np.vstack((repstack, su, fd2, fp2, fi2)).T

100%|██████████| 500/500 [01:43<00:00,  4.81it/s]


In [3]:
from pprint import pprint
print(result.shape)
pprint(result)

(34047, 5)
array([[1.00000000e+00, 3.20000000e+01, 0.00000000e+00, 9.53834414e-07,
        0.00000000e+00],
       [1.00000000e+00, 2.12000000e+02, 4.76836855e-06, 4.76917207e-06,
        5.72204590e-06],
       [1.00000000e+00, 2.13000000e+02, 1.58945618e-06, 0.00000000e+00,
        9.53674316e-07],
       ...,
       [5.00000000e+02, 3.31100000e+03, 1.74840180e-05, 9.53834414e-07,
        1.90734863e-06],
       [5.00000000e+02, 3.31300000e+03, 3.97364046e-05, 2.67073636e-05,
        3.24249268e-05],
       [5.00000000e+02, 3.32000000e+03, 1.58945618e-06, 1.90766883e-06,
        9.53674316e-07]])


In [4]:
import pandas as pd
import h5py, re, sys
import numpy as np
from scipy.stats import spearmanr
from scipy.interpolate import UnivariateSpline
from pylab import *

threshold = 0.02
orders = np.arange(2, 14)
min_order, max_order = 2, 13


pp = []
r = np.arange(1, 501)

with pd.HDFStore("PI.h5") as store:
    for onum in orders:
        # f=open('pp'+str(onum),"w")
        o = "/order" + str(onum)
        # if d.data>0:
        gg = store[o]["data"]
        gg2 = store[o]["indep"]
        gg3 = store[o]["model"]
        gg4 = store[o]["repnum"]

        g = np.vstack((gg, gg2, gg3, gg4)).T
        # g=g[g[:,0]>threshold]
        gl = g[np.argsort(-g[:, 0])]
        h = gl[gl[:, 0] > threshold]

        # h=[gl[gl[:,3]==rr][1] for rr in r]
        # h=np.asarray(h)
        if onum == min_order:
            gl_o = gl
            h_o = h
        pp.append([spearmanr(h[:, 0], h[:, 1])[0], spearmanr(h[:, 0], h[:, 2])[0]])
        # pp.append([spearmanr(gl[:,0],gl[:,1])[0],spearmanr(gl[:,0],gl[:,2])[0]])

pp = np.asarray(pp)

In [7]:
result

array([1.00000000e+00, 3.13000000e+02, 3.17891237e-06, 9.53834414e-07,
       9.53674316e-07])